[View in Colaboratory](https://colab.research.google.com/github/ZZANZU/DeeeeeepLearning/blob/master/cnn_mnist.ipynb)

In [89]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### GPU 작동 여부 체크

In [90]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [91]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10243264095592255613, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281989632
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16149702693122345365
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

## CNN 계층 구성

In [0]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # 커널 크기를 2X2로 해서 2칸씩 움직임, 14X14 됨

In [0]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # 7X7 됨

In [0]:
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # 3X3 됨?

In [0]:
W4 = tf.Variable(tf.random_normal([3 * 3 * 128, 256], stddev=0.01))
L4 = tf.reshape(L3, [-1, 3 * 3 * 128]) # -1 : 고정된 차원은 우선 채우고 남은 부분은 알아서 채워준다.
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, keep_prob) # 1X256

In [0]:
W5 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L4, W5) # hypothesis

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

## 모델 학습

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [113]:
with tf.device('/gpu:0') :
    for epoch in range(15) :
        total_cost = 0

        for i in range(total_batch) :
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1) # import할 때 784

            _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs,
                                                                                            Y: batch_ys,
                                                                                            keep_prob: 0.7})
            total_cost += cost_val
        
        print('Epoch:', '%04d' % (epoch+1), 'Avg. cost =', '{:3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

InvalidArgumentError: ignored

In [0]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), 
                                                                        Y: mnist.test.labels,
                                                                        keep_prob: 1}))